# Modelo de classificação de texto utilizando Machine Learning
O objetivo é criar um modelo de machine learning capaz de classificar o textos de e-mail, sms, ou messageiros como sendo FRAUDE ou NÃO FRAUDE


In [0]:
# Importando as bibliotecas necessarias
import numpy as np
import pandas as pd
import nltk
import string
from google.colab import files

In [0]:
# Carega o dataset a partir do computador
#uploaded = files.upload()

In [0]:
# Caso esteja utilizando o Google Colab você pode montar o Google Drive para pegar o arquivo do dataset salvo
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Sobre o Dataset
No momento em que fiz este modelo não encontrei um dataset pronto, então tive que criar um.
Felizmente encontrei o site https://catalogodefraudes.rnp.br/ que contém um repositório de mensagens classificadas como fraudulentas.
O que fiz então foi criar um Crawler para coletar as informções necessarias para criar meu dataset.

In [0]:
# Lê o arquivo do dataset em formato CVS - É necessário passar o caminho do arquivo
dataset_pd = pd.read_csv('/content/drive/My Drive/Colab Notebooks/fraude.csv',encoding='utf-8')

In [0]:
# Cria o DataFrame utilizando o Pandas
# Para este modelo eu terei que utilizar 2 dataframes, então este é o primeiro a ser criado
df1 = pd.DataFrame(dataset_pd)

In [0]:
# Exibe as 5 primeiras linhas do dataframe
df1.head()

,_id,title,assunto,conteudo
0,13889,Nota Fiscal Eletrônica,Portal de Documentos Fiscais Brasil - Y4GXJFF9...,Portal de Documentos Fiscais Brasil\r\n\r\n Re...
1,13887,Currículo,Curriculum Vitae,"Boa tarde, Segue em anexo meu curriculum, tenn..."
2,13888,Dispositivo de Segurança Bradesco,Dispositivo de Seguranca Desabilitado - Conta ...,Informamos que o período de uso do seu disposi...
3,13885,Fatura Net,Fatura em atraso,"Para a NET, é muito importante ser transparent..."
4,13886,Atualização de e-mail,Aviso de e-mail,"Querido usuário,\r\n\r\nSua conta de e-mail fo..."


Observando o dataset temos os campos _id, title, assunto, conteudo
Neste caso irei utilizar somente o campo conteudo, ou seja, somente o texto da mensagem.
Também é necessario adicionar o campo fraude para saber qual texto é fraude, neste caso todas são fraude por isso eu adiciono a coluna fraude com o valor 1 em todas as linhas

In [0]:
# Adiciona o coluna 'fraude' com o valor 1
df1['fraude'] = 1

In [0]:
# Exibe o cabeçalho com as novas alterações
df1.head()

,_id,title,assunto,conteudo,fraude
0,13889,Nota Fiscal Eletrônica,Portal de Documentos Fiscais Brasil - Y4GXJFF9...,Portal de Documentos Fiscais Brasil\r\n\r\n Re...,1
1,13887,Currículo,Curriculum Vitae,"Boa tarde, Segue em anexo meu curriculum, tenn...",1
2,13888,Dispositivo de Segurança Bradesco,Dispositivo de Seguranca Desabilitado - Conta ...,Informamos que o período de uso do seu disposi...,1
3,13885,Fatura Net,Fatura em atraso,"Para a NET, é muito importante ser transparent...",1
4,13886,Atualização de e-mail,Aviso de e-mail,"Querido usuário,\r\n\r\nSua conta de e-mail fo...",1


In [0]:
# Remove as colunas que não serão utilizadas ('_id', 'title' e 'assunto')
df1.drop(['_id', 'title','assunto'],axis=1, inplace=True)

In [0]:
# Exibe o cabeçalho com as novas alterações
df1.head()

,conteudo,fraude
0,Portal de Documentos Fiscais Brasil\r\n\r\n Re...,1
1,"Boa tarde, Segue em anexo meu curriculum, tenn...",1
2,Informamos que o período de uso do seu disposi...,1
3,"Para a NET, é muito importante ser transparent...",1
4,"Querido usuário,\r\n\r\nSua conta de e-mail fo...",1


In [0]:
df1.shape

(10449, 2)

In [0]:
# Aqui estou mantendo apenas 6000 linhas do meu dataframe, pois como veremos mais a frente
# o meu segundo dataset que contém as mensagens que não são fraude contem apenas 4212
# apos a limpesa do primeiro dataframe ele ficará com 4675
df1 = df1[:6000]

In [0]:
df1.shape

(6000, 2)

In [0]:
# Remove os conteudos duplicados
df1.drop_duplicates(inplace=True)

In [0]:
# Remove os conteudos nulos
df1.isnull().sum()

conteudo    1
fraude      0
dtype: int64

In [0]:
# Mostra a quantidade de linhas e colulas do dataFrame 1 depois da alteração
df1.shape

(4675, 2)

In [0]:
# Baixa a lista de 'stopwords' e seta o "dicionário" portugues
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Função que é utilizada para "limpar" o texto
def process_text_conteudo(conteudo):
  import unicodedata

  # Remove pontuação
  nopont = [char for char in conteudo if char not in string.punctuation]

  # Forma as palavras todas em minúnculas
  nopont = ''.join(nopont).lower()
  
  # remove acentos
  nopont = unicodedata.normalize('NFKD', nopont).encode('ASCII', 'ignore').decode('ASCII')

  # cria lista contendo somente palavras removento as stopwords
  clean_palavras = [palavra for palavra in nopont.split() if palavra not in stopwords]

  return clean_palavras


In [0]:
# Mostra o campo 'conteudo' apos filtar as palavras
df1['conteudo'].head().apply(process_text_conteudo)

0    [portal, documentos, fiscais, brasil, referenc...
1    [boa, tarde, segue, anexo, curriculum, tennho,...
2    [informamos, periodo, uso, dispositivo, segura...
3    [net, importante, ser, transparente, voce, inf...
4    [querido, usuario, conta, email, login, local,...
Name: conteudo, dtype: object

# Segundo Dataset
Este dataset foi criado a partir de menssagens pessoais de e-mail.
Na criação deste dataset eu já realizei as alterações necessarias, mantive somente o texto da mensagem e adicionei a coluna fraude com o valor 0 em todas as linhas

In [0]:
# Carega o dataset do PC
#uploaded = files.upload()

In [0]:
# Lê o arquivo cvs
dataset_pd_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset_emails.csv',encoding='utf-8')

In [0]:
# Cria o segundo DataFrame
df2 = pd.DataFrame(dataset_pd_2)

In [0]:
# Exibe o cabeçalho com (5 primeiras linhas) do 2 dataframe
df2.head()

,conteudo,fraude
0,\n\n\n\n\n \n\t\t\tHoje no Twitter\n\t\nEm men...,0
1,\n\n\n\n\n\nIMG_1110401321946.jpeg\n\n\nIMG_11...,0
2,\n\n\n\n\nContas Firefox\nAutenticação em duas...,0
3,\n\n\n\n\n \n \nLinkedIn \t\n \n\tGrupos\n \n ...,0
4,"\n\n\n\n\nOlá Jones,\n\nO nosso aluno, O , pr...",0


In [0]:
# Remove os conteudos nulos
df2.isnull().sum()

conteudo    0
fraude      0
dtype: int64

In [0]:
# Remove os conteudos duplicados
df2.drop_duplicates(inplace=True)

In [0]:
# Mostra a quantidade de linhas e colulas do dataFrame 2 depois da alteração
df2.shape

(4212, 2)

In [0]:
# Mostra o campo 'conteudo' apos filtrar as palavras
df2['conteudo'].head().apply(process_text_conteudo)
# a função process_text_conteudo já foi criada acima

0    [hoje, twitter, mensagem, natal, papa, condena...
1    [img1110401321946jpeg, img1110401321946jpeg, 6...
2    [contas, firefox, autenticacao, duas, etapas, ...
3    [linkedin, grupos, seguranca, informacao, segu...
4    [ola, jones, aluno, precisa, fazer, prova, sub...
Name: conteudo, dtype: object

In [0]:
# Criando um novo dataframe com os 2 dataframes
data_frames = [df1, df2]

In [0]:
# Concatena os 2 dataframes
data_frame = pd.concat(data_frames)

In [0]:
# Mostra o corpo do novo dataset
data_frame.shape

(8887, 2)

In [0]:
# Mostra o cabeçalho do dataframe
data_frame.head

<bound method NDFrame.head of                                                conteudo  fraude
0     Portal de Documentos Fiscais Brasil\r\n\r\n Re...       1
1     Boa tarde, Segue em anexo meu curriculum, tenn...       1
2     Informamos que o período de uso do seu disposi...       1
3     Para a NET, é muito importante ser transparent...       1
4     Querido usuário,\r\n\r\nSua conta de e-mail fo...       1
...                                                 ...     ...
4553  \n\n\n\n\n\n \n \nLinkedIn\t\n \n\tGrupos\n \n...       0
4554  \n\n\n\n\nHostGator Brasil\n\nOlá  de  ,\n\nRe...       0
4555  \n\n\n\n\n\nFacebook\nfacebook\nPessoas que vo...       0
4556  \n\n\n\n\n\nOlá, , tudo bem?\n\n\n\n\n\n\n\nCO...       0
4557       \n\n\n\n\n\n\nPortal Poupatempo.pdf\t84,2 KB       0

[8887 rows x 2 columns]>

In [0]:
# Transforma as palavras em uma matriz de numeros tokens
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=process_text_conteudo)
vect = vectorizer.fit_transform(data_frame['conteudo'].values.astype('U'))

In [0]:
# Exibe o corpo da matiz
print(vect.shape)


(8887, 55601)


In [0]:
from sklearn.model_selection import train_test_split
# Separando dados em 80% treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(vect, data_frame['fraude'] , test_size=0.20, random_state=0)

In [0]:
from sklearn.naive_bayes import MultinomialNB
# Criando modelo de classificação com algoritimo naive_bayes
classifier = MultinomialNB().fit(X_train, y_train)

In [0]:
# Relatorio e estatisticas com os dados do treinamento
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train, pred))
print()
print('Confusion Matrix: \n',confusion_matrix(y_train, pred))
print()
print('Accuracy: ', accuracy_score(y_train, pred))


              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3387
           1       0.97      0.99      0.98      3722

    accuracy                           0.98      7109
   macro avg       0.98      0.98      0.98      7109
weighted avg       0.98      0.98      0.98      7109


Confusion Matrix: 
 [[3281  106]
 [  51 3671]]

Accuracy:  0.9779153186102124


In [0]:
# Relatorio e estatisticas com os dados do teste
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test, pred))
print()
print('Confusion Matrix: \n',confusion_matrix(y_test, pred))
print()
print('Accuracy: ', accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       825
           1       0.96      0.98      0.97       953

    accuracy                           0.97      1778
   macro avg       0.97      0.97      0.97      1778
weighted avg       0.97      0.97      0.97      1778


Confusion Matrix: 
 [[790  35]
 [ 19 934]]

Accuracy:  0.96962879640045


In [0]:
#exporta o modelo 
import pickle
filenameModelo = 'modelo.pk1'
pickle.dump(classifier, open(filenameModelo, 'wb'))


In [0]:
#exporta o dictionary
filenameMessagesWords = 'messages_words.pk1'
pickle.dump(vectorizer.vocabulary_, open(filenameMessagesWords, 'wb'))


# Teste de aplicação
Aqui inicia o teste "real" do modelo criado

In [0]:
import os
import pickle
import numpy as np
import pandas as pd
import sklearn.feature_extraction

In [0]:
#uploaded = files.upload()

In [0]:
# Abre o modelo e o dicionário
modelo = pickle.load(open('/content/modelo.pk1','rb'))
modelo_messages_words = pickle.load(open('/content/messages_words.pk1','rb'))

In [0]:
# Exemplo de Email
# O primeio exemplo é um e-mail normal
dados = {"conteudo":"Olá, pessoal, pronto para um novo desafio DEV? Hoje sai o terceiro vídeo da semana devepoler. Lembrando que esta é uma aula para todos os níveis de conhecimento. Ou seja, se você está no nível básico, intermediário ou avançado, a aula será bem produtiva e você aprenderá muito sobre vários temas. Como Nodejs, MongoDB, VueJS e muito mais: esta aula ficará no ar somente durante 7 dias, então recomendo assistir agora mesmo, pois se você deixar para depois, ela poderá não estar mais disponível!  Equipe xyz"}
# Exemplo de email FRAUDE
#dados = {"conteudo":"Netflix Atualize seus dados de pagamento! Olá <email> Ocorreu um problema com seu meio de pagamento atual. Tentaremos efetuar uma cobrança novamente mas recomendamos que você atualize essas informações. ATUALIZAR PAGAMENTO Lembrando que sua conta estará suspensa caso o débito não seja realizado. Informações da sua conta: Email <email> Provedor do serviço Netflix Entretenimento Brasil, Ltda. Plano Básico Preço do plano R$21,90/mês Estamos sempre prontos para ajudar. Acesse o Centro de ajuda para saber mais ou fale com a gente. Seus amigos da Netflix VEJA TODAS AS SÉRIES E FILMES> Ao assinar a Netflix, você concorda com nossos Termos de uso e com a Declaração de privacidade Dúvidas? Ligue 0800-887-0201 Estamos enviando este email porque você é assinante Netflix. Para alterar suas preferências de email a qualquer momento, acesse a página Configurações de comunicação da sua conta. Não responda a este email, pois a caixa deste endereço de email não é monitorada. Para obter ajuda ou entrar em contato, acesse o Centro de ajuda em help.netflix.com. Esta mensagem foi enviada para [ erik1605@hotmail.com  ] pela Netflix. SRC: 12500_pt-BR_BR A utilização do serviços e do site da Netflix está sujeita aos Termos de privacidade. Netflix Entretenimento Brasil, Ltda."}

In [0]:
# Processa o texto chamando a função process_text_conteudo para limpar o texto
text_process = process_text_conteudo(dados.values())

In [0]:
# Carrega o modelo passando o dicionário
loaded_vectorizer = sklearn.feature_extraction.text.CountVectorizer(vocabulary=modelo_messages_words)

In [0]:
# Faz a predição do texto
predicao = modelo.predict(loaded_vectorizer.transform(text_process))

In [0]:
# Resultado da predição usando os exemplos criados
resultado = predicao[0]
# Se for Fraude imprime FRAUDE se não imprime OK
resposta = 'FRAUDE' if resultado == 1 else 'OK'
print(resposta)

OK
